# Intensity analysis diagrams
In Python.
- **Input**: The three intensity analysis levels data files: "IIA.csv" (Interval level), "CIA.csv" (Category level), "TIA.csv" (Transition level). 
- **Output**: Plots for the three intensity analysis levels.

In [ ]:
import csv
import numpy as np
import plotly.graph_objects as go
import plotly
import matplotlib.pyplot as plt
from matplotlib import font_manager

In [ ]:
xticks_font = font_manager.FontProperties(family='serif')

colors = {'Extremely Low': 'yellow',
          'Low' : "#ffa500",
          'Moderate' : "brown",
          'High' : "#90ee90",
          'Very High' : "green",}
categories = ['Extremely Low', 'Low', 'Moderate', 'High', 'Very High']
nb_categories = len(categories)

year_format=[]
length_inteval = 4
start_year = 2002
end_year = 2022
nb_intervals = int((end_year - start_year) / length_inteval)
for i in range(nb_intervals):
    year_format.append(f"{start_year+i*length_inteval}\n{start_year+(i+1)*length_inteval}")

## Interval-level Intensity Analysis
Displays change for each interval:
- **Time interval change size (km²):** For each time interval, the total area (in km²) of change is displayed.
- **Annual change intensity (%):** For each interval, the sum of element changes is devided by the total number of elements (element = pixel or km²). The obtained result is then devided by 2 in order to obtain the annual change intensity.
- **Uniform change across intervals:** The vertical black line. Represents the intensities mean. Below it, the change in the interval is considered as slow. Above it, it is considered as fast. 

In [ ]:
#Reading the IIA data
filename = 'IIA.csv'
with open(filename) as f:
    reader = csv.reader(f)
    data = []
    for r in reader:
        data.append(r)
    
change=[]
intensity=[]

for i in range(nb_intervals):
    change.append(float(data[i+1][1])*0.0009)
    intensity.append(float(data[i+1][2]))
uniform = float(data[1][3])

# Set the figure size
plt.rcParams["figure.figsize"] = [12, 3]

# Array for horizontal bar's position
ind = np.array(range(nb_intervals))

# Bar's width
width = 0.3

fig, axs = plt.subplots(1, 2)

# Annual change
axs[0].barh(ind,change,width,color='grey',edgecolor='black')
axs[0].set(yticks=ind, yticklabels=[])
for tick in axs[0].get_xticklabels():
    tick.set_fontproperties(xticks_font)
axs[0].invert_xaxis()
axs[0].tick_params(
    axis='y',          # changes apply to the y-axis
    which='both',      # both major and minor ticks are affected
    left=False,        # ticks along the bottom edge are off
    labelbottom=False)  
axs[0].set_xticks(ticks=np.arange(0, 180000, step=40000),
                  labels=np.arange(0, 180000, step=40000),font="serif")
axs[0].set_facecolor("white")
axs[0].grid(axis='x',color = 'grey', linestyle = '--', linewidth = 0.5)
axs[0].grid(axis='y', color = 'grey', linestyle = '--', linewidth = 0.5)
axs[0].set_title(f"Time interval change size (km²)",fontsize = 12,font="serif") 
    
# Annual change intensity
axs[1].barh(ind, intensity, width, color='gray', edgecolor='black')
        
for tick in axs[1].get_xticklabels():
    tick.set_fontproperties(xticks_font)
axs[1].set(yticks=ind)
axs[1].set_yticklabels(np.array(year_format),fontsize=12,font="serif")    
axs[1].tick_params(
    axis='y',          # changes apply to the y-axis
    which='both',      # both major and minor ticks are affected
    left=False,        # ticks along the bottom edge are off
    labelbottom=False)  
        
axs[1].set_facecolor("white")
axs[1].grid(axis='x',color = 'grey', linestyle = '--', linewidth = 0.5)
axs[1].grid(axis='y', color = 'grey', linestyle = '--', linewidth = 0.5)

axs[1].set_title(f"Annual change intensity (%)",fontsize = 12,font="serif")
    
#Uniform category intensity lines
point1 = [uniform,0 - width/2]
point2 = [point1[0], point1[1] + nb_intervals-1 + width ]
x_values = [point1[0], point2[0]]
y_values = [point1[1], point2[1]]
axs[1].plot(x_values, y_values, "black", linestyle="-")
axs[1].text(point1[0] - 0.014 ,  point2[1], "slow",fontsize = 8,font="serif")
axs[1].text(point1[0] + 0.003 ,  point2[1], "fast",fontsize = 8,font="serif")

# Save the plot (dimension shoukd be adapted)
# plt.rcParams.update({'font.size': 12})
# plt.savefig("iia.png")

## Category-level Intensity Analysis :
Displays gain and loss for each category in each interval :
- **Annual change size (km²):** For each time interval, the area (in km²) of gain and loss are displayed for each category. The obtained numbers are divided by 2 in order to obtain annual changes.
- **Annual change intensity (%):** For each interval, the number of gain and loss elements for a category are devided by the number of elements for this catogory in the interval starting year (element = pixel or km²). The obtained result is then devided by 2 in order to have the annual change intensity by category.
- **Uniform category intensity:** The black vertical line. Represents the annual change intensity for the interval. Below it, the gain or loss change is considered as dormant. Above it, it is considered as active. 

In [ ]:
#Reading the CIA data
filename = 'CIA.csv'
with open(filename) as f:
    reader = csv.reader(f)
    data = []
    for r in reader:
        data.append(r)
    
gross_loss=[[] for _ in range(nb_intervals)]
gross_gain=[[] for _ in range(nb_intervals)]
intensity_loss=[[] for _ in range(nb_intervals)]
intensity_gain=[[] for _ in range(nb_intervals)]
uniform=[]

for i in range(nb_intervals):
    for c in range(nb_categories):
        gross_loss[c].append(float(data[2+i*8+c][1])*0.0009)
        gross_gain[c].append(float(data[2+i*8+c][2])*0.0009)
        intensity_loss[c].append(float(data[2+i*8+c][3]))
        intensity_gain[c].append(float(data[2+i*8+c][4]))
    uniform.append(float(data[2+i*8+c][5]))

labels = []
for c in categories:
    labels.append(f"{c} Loss")
    labels.append(f"{c} Gain")

# Set the figure size
plt.rcParams["figure.figsize"] = [12, 4]
#plt.rcParams["figure.autolayout"] = True

# Array for horizontal bar's position
ind = np.array(range(nb_intervals))

# Bar's width
width = 0.08

fig, axs = plt.subplots(1, 2)

l = []

# Annual change area by category
for c in range(nb_categories):
    l.append(axs[0].barh(ind + width*(2*c), gross_loss[c], width, color=colors[categories[c]], hatch="//",edgecolor='black')[0])
    l.append(axs[0].barh(ind + width*(2*c+1), gross_gain[c], width, color=colors[categories[c]],edgecolor='black')[0])
        
    # Set Y-axis ticks and ticklabels
    axs[0].set(yticks=ind+width*4.5, yticklabels=[])
    for tick in axs[0].get_xticklabels():
        tick.set_fontproperties(xticks_font)
    axs[0].set_xticks(ticks=np.arange(0, 23000, step=5000),labels=np.arange(0, 23000, step=5000),font="serif")

    axs[0].tick_params(
        axis='y',          # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left=False,        # ticks along the bottom edge are off
        labelbottom=False)  
    
    axs[0].invert_xaxis()
    
    axs[0].set_facecolor("white")
    axs[0].grid(axis='x',color = 'grey', linestyle = '--', linewidth = 0.5)
    axs[0].grid(axis='y', color = 'grey', linestyle = '--', linewidth = 0.5)
    
    axs[0].set_title(f"Annual change size (km²)",fontsize = 12,font="serif")
    
# Annual change intensity
for c in range(5):
    axs[1].barh(ind + width*(2*c), intensity_loss[c], width, color=colors[categories[c]], hatch="//",edgecolor='black')
    axs[1].barh(ind + width*(2*c+1), intensity_gain[c], width, color=colors[categories[c]],edgecolor='black')
        
    for tick in axs[1].get_xticklabels():
        tick.set_fontproperties(xticks_font)
    axs[1].set(yticks=ind+width*4.5)
    axs[1].set_yticklabels(np.array(year_format),fontsize=12,font="serif")    
    axs[1].tick_params(
        axis='y',          # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left=False,        # ticks along the bottom edge are off
        labelbottom=False)  
        
    axs[1].set_facecolor("white")
    axs[1].grid(axis='x',color = 'grey', linestyle = '--', linewidth = 0.5)
    axs[1].grid(axis='y', color = 'grey', linestyle = '--', linewidth = 0.5)

    axs[1].set_title(f"Annual change intensity (%)",fontsize = 12,font="serif")
    
#Uniform category intensity lines
for i in range(nb_intervals):
    point1 = [uniform[i], -0.5*width + ind[i] ]
    point2 = [point1[0], point1[1] + 10*width ]
    x_values = [point1[0], point2[0]]
    y_values = [point1[1], point2[1]]
    axs[1].plot(x_values, y_values, "black", linestyle="-")
    axs[1].text(point1[0] - 0.03 ,  point1[1] + width * 10 + 0.01, "dormant",fontsize = 8,font="serif")
    axs[1].text(point1[0] + 0.003 ,  point1[1] + width * 10 + 0.01, "active",fontsize = 8,font="serif")

fig.legend(l, labels, loc='upper center', bbox_to_anchor=(0.51,1.1), ncol=5, prop={"family":"Serif","size":8})

# Save the plot (dimension shoukd be adapted)
# plt.rcParams.update({'font.size': 12})
# plt.savefig("cia.png")

## Transition-level Intensity Analysis:
Displays gains of each category from every other category in each interval:
- **Annual transition area for gain (km²):** For each time interval, the transition area (in km²) for gain of each category from each other category is displayed. The obtained numbers are divided by 2 in order to obtain annual changes.
- **Annual transition intensity for gain (percentage):** For each interval, the number of elements transitioning to a category from another are devided by the number of elements of the second catogory at the starting year of the interval (element = pixel or km²). The obtained result is then devided by 2 in order to have the annual intensity change by transition.
- **Uniform Transition Intensity:** The red dashed line. Reprsents the total number of elements gained by the category devided by the total number of elements of all the other categories at the starting year of the interval. A category with a change intensity below it, is considered as avoided. Otherwise, it is considered targeted.

**Note**: Plot scales (same or adaptive) can be configured through the boolean variable "same_scale". 

In [ ]:
nb_categories_to_display = 5
#All plots have the same scale (True). Every plot is scaled according to its max value (False).
same_scale = True

filename = 'TIA.csv'
with open(filename) as f:
    reader = csv.reader(f)
    data = []
    for r in reader:
        data.append(r)

other_categories=[[[] for _ in range(nb_categories-1)] for _ in range(nb_categories_to_display)]
transition=[[[] for _ in range(nb_categories-1)] for _ in range(nb_categories_to_display)]
transition_intensity=[[[] for _ in range(nb_categories-1)] for _ in range(nb_categories_to_display)]
uniform=[[[] for _ in range(nb_categories-1)] for _ in range(nb_categories_to_display)]

#main categories
for c in range(nb_categories_to_display):
    #intervals
    for i in range(nb_intervals):
        #other categories
        for oc in range(nb_categories-1):
            other_categories[c][oc].append(data[2+oc+i*31+c*6][0])
            transition[c][oc].append(float(data[2+oc+i*31+c*6][1])*0.0009)            
            transition_intensity[c][oc].append(float(data[2+oc+i*31+c*6][2]))            
            uniform[c][oc].append(float(data[2+oc+i*31+c*6][3]))

# Set the figure size
plt.rcParams["figure.figsize"] = [12, 18]
# plt.rcParams["figure.autolayout"] = True

# Array for horizontal bar's position
ind = np.array(range(nb_intervals))

# Bar's width
width = 0.2

fig, axs = plt.subplots(nb_categories_to_display, 2)

L = []

for c in range(nb_categories_to_display):
    for oc in range(nb_categories-1):
        l = axs[c,0].barh(ind + width*oc, transition[c][oc], width, color=colors[other_categories[c][oc][0]],edgecolor='black')[0]
        L.append(l)
        
    # Set Y-axis ticks and ticklabels
    axs[c,0].set(yticks=ind+width*1.5, yticklabels=[])
        
    axs[c,0].tick_params(
        axis='y',          # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left=False,       # ticks along the bottom edge are off
        labelbottom=False)  
    
    if same_scale :
        axs[c,0].set_xticks(ticks=np.arange(0, 18000, step=4000),labels=np.arange(0, 18000, step=4000),font="serif")
    
    for tick in axs[c,0].get_xticklabels():
        tick.set_fontproperties(xticks_font)

    axs[c,0].invert_xaxis()
    
    axs[c,0].set_facecolor("white")
    axs[c,0].grid(axis='x',color = 'grey', linestyle = '--', linewidth = 0.5)
    axs[c,0].grid(axis='y', color = 'grey', linestyle = '--', linewidth = 0.5)

#main categories
for c in range(nb_categories_to_display):
    #other categories
    for oc in range(nb_categories-1):
        l = axs[c,1].barh(ind + width*oc, transition_intensity[c][oc], width, color=colors[other_categories[c][oc][0]],edgecolor='black')[0]
        L.append(l)
    
    axs[c,1].set_facecolor("white")
    axs[c,1].grid(axis='x', color = 'grey', linestyle = '--', linewidth = 0.5)
    axs[c,1].grid(axis='y', color = 'grey', linestyle = '--', linewidth = 0.5)
    
    # Set Y-axis ticks and ticklabels
    axs[c,1].set(yticks=ind+width*1.5, yticklabels=np.array(year_format))
    
    if same_scale:
        axs[c,1].set(xlim=[0,0.16])
                     
    axs[c,1].set_yticklabels(np.array(year_format),fontsize=12,font="serif")
    axs[c,1].set_xticks(ticks=np.array(np.arange(0, 0.16, step=0.03)),
                        labels=np.array(np.arange(0, 0.16, step=0.03)),
                        fontsize=12)
    for tick in axs[c,1].get_xticklabels():
        tick.set_fontproperties(xticks_font)
    
    axs[c,1].tick_params(
        axis='y',          # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left=False,        # ticks along the bottom edge are off
        labelbottom=False)  

for c in range(nb_categories_to_display):
    for i in range(nb_intervals):
        point1 = [uniform[c][0][i], -width/2 + ind[i] ]
        point2 = [point1[0], point1[1] + 4*width ]
        x_values = [point1[0], point2[0]]
        y_values = [point1[1], point2[1]]
        axs[c,1].plot(x_values, y_values, "black", linestyle="-")
        point2_ax = axs[c,1].transLimits.transform(point2)
        axs[c,1].text(point2[0]/axs[c,1].get_xlim()[1]-0.08,point2_ax[1], "avoid",fontsize = 8,transform=axs[c,1].transAxes,font="serif",)
        axs[c,1].text(point2[0]/axs[c,1].get_xlim()[1]+0.01,point2_ax[1], "target",fontsize = 8,transform=axs[c,1].transAxes,font="serif",)
        
fig.legend((L[4],L[0],L[1],L[2],L[3]), (categories[0],categories[1],categories[2],categories[3],categories[4]), 
    loc='upper center', bbox_to_anchor=(0.5,0.94), ncol=5,fontsize = 12, prop={"family":"Serif"})
  
#Adjust space between subplots      
fig.subplots_adjust(wspace=0.15)

#Common titles to columns and rows
cols = ['Annual transition size (km²)','Annual transition intensity (%)']
pad = 0 # in points
for ax, col in zip(axs[0], cols):
    ax.annotate(col, xy=(0.5, 1.05), xytext=(0, pad),
                xycoords='axes fraction', textcoords='offset points',
                size=12, ha='center', va='baseline', font="serif" )
for ax, row in zip(axs[:,0], categories[0:nb_categories_to_display]):
    ax.annotate(row, xy=(0, 0.5), xytext=(5, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size=12, ha='right', va='center', font="serif", rotation=90)